In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from scipy.signal import lfilter, butter

import os
import glob

from tqdm import tqdm
%matplotlib inline

# Upload the data 

In [2]:
path = r'C:\Users\ecath\Desktop\Research\Raw Data\Ex Vivo CSV Files'
os.listdir(path)

['H72_F75_Electrode_KH_OM_KH.csv',
 'H79_F108_csv_annot_OM_KH.csv',
 'H79_F125_csv_annot MISSING H8_OM_KH.csv',
 'H81_F158_csv_annot_KH.csv',
 'H81_F171_csv_annot_OM_KH.csv',
 'H82_F111_csv_annot_OM_KH.csv',
 'H96_F61_csv_annot_OM_KH.csv',
 'H96_F88_csv_annot_OM_KH.csv']

In [3]:
data_list = []
for filename in glob.glob(os.path.join(path, '*.csv')):
    if filename != r'C:\Users\ecath\Desktop\Research\Raw Data\Ex Vivo CSV Files\H72_F75_Electrode_KH_OM_KH.csv':
        data = pd.read_csv(filename, header=0)
        print(filename, data.shape)
        data_list.append(data)

C:\Users\ecath\Desktop\Research\Raw Data\Ex Vivo CSV Files\H79_F108_csv_annot_OM_KH.csv (8270, 384)
C:\Users\ecath\Desktop\Research\Raw Data\Ex Vivo CSV Files\H79_F125_csv_annot MISSING H8_OM_KH.csv (8775, 384)
C:\Users\ecath\Desktop\Research\Raw Data\Ex Vivo CSV Files\H81_F158_csv_annot_KH.csv (8138, 384)
C:\Users\ecath\Desktop\Research\Raw Data\Ex Vivo CSV Files\H81_F171_csv_annot_OM_KH.csv (10173, 384)
C:\Users\ecath\Desktop\Research\Raw Data\Ex Vivo CSV Files\H82_F111_csv_annot_OM_KH.csv (10173, 384)
C:\Users\ecath\Desktop\Research\Raw Data\Ex Vivo CSV Files\H96_F61_csv_annot_OM_KH.csv (8191, 384)
C:\Users\ecath\Desktop\Research\Raw Data\Ex Vivo CSV Files\H96_F88_csv_annot_OM_KH.csv (8191, 384)


# Division by classes 

In [4]:
drivers = []
nondrivers = []
noises = []

for df in data_list:
    driver = pd.DataFrame()
    nondriver = pd.DataFrame()
    noise = pd.DataFrame()
    for col in df.columns[1::3]:
        curr_index = list(df.columns).index(col)
        prev_index = curr_index - 1
        next_index = curr_index + 1
        prev_col = df.iloc[:,prev_index]
        next_col = df.iloc[:,next_index]
        if df[col][0] == 1:
            driver[df.columns[prev_index]] = prev_col
            driver[df.columns[next_index]] = next_col
        elif df[col][0] == 0:
            nondriver[df.columns[prev_index]] = prev_col
            nondriver[df.columns[next_index]] = next_col
        elif df[col][0] == -1:
            noise[df.columns[prev_index]] = prev_col
            noise[df.columns[next_index]] = next_col
    drivers.append(driver)
    nondrivers.append(nondriver)
    noises.append(noise)

# Butterworth highpass for 1Hz

In [5]:
def butter_highpass(data, highcut, fs, order=2):
    
    nyq = 0.5 * fs
    high = highcut / nyq
    b, a = butter(order, high, btype='highpass', analog=False)
    y = lfilter(b, a, data)
    
    return y

# Fourier transform

In [6]:
# function that delete NaN and fully zero columns
def del_nul_and_nan(df):
    
    df.fillna(value=0, axis=1, inplace=True)
    
#     for col in df.columns:
#         if df[col].sum() == 0:
#             df.drop([col], axis = 1, inplace = True)  
    
    return df

In [7]:
#create DataFrame with signal spectrum cutted on 20Hz for one dataframe from one class

def concat_fft(df, N, F, low_freq=1.0):
    
    fft_out = pd.DataFrame()
    
    for col in df.columns:
        filtered_yf = butter_highpass(df[col], low_freq, F, order=2)
        fft_yf = np.fft.fft(filtered_yf) #spectrum
        fft_xf = np.fft.fftfreq(N, 1/F) #frequencies
        
        fft_20_index = np.argwhere((fft_xf<20) & (fft_xf>0))        
        fft_yf_20 = fft_yf[fft_20_index] #cutting on 20Hz
        fft_xf_20 = fft_xf[fft_20_index] #cutting on 20Hz
        
        fft_yf_20 = pd.DataFrame(np.abs(fft_yf_20)/N, columns=[col + '_yf'])
        fft_xf_20 = pd.DataFrame(fft_xf_20, columns=[col + '_xf']) 

        fft_out = pd.concat([fft_out, fft_yf_20, fft_xf_20], axis=1)
        
    return(fft_out)

In [52]:
def full_fft_list(class_df):
    
    all_fft_el = []
    all_fft_om = []

    for k, df in enumerate(class_df):

        electrode_signal = df[df.columns[::2]]
        optical_signal = df[df.columns[1::2]]

        electrode_signal = del_nul_and_nan(electrode_signal)
        optical_signal = del_nul_and_nan(optical_signal)        

        N = df.shape[0] # Number of sample points
        Fs_el = 1017.25 # sampling rate
        Fs_om = 1000.0

        fft_el = concat_fft(electrode_signal, N, Fs_el, low_freq=1.0)
        fft_om = concat_fft(optical_signal, N, Fs_om, low_freq=1.0)
        
        all_fft_el.append(fft_el)
        all_fft_om.append(fft_om)
        
    return(all_fft_el, all_fft_om)

In [11]:
def fft_creation_el(dataframe, label):
    
    all_fft_el, _ = full_fft_list(dataframe)
    
    m = []
    
    for i in range(len(all_fft_el)):
        m.append(all_fft_el[i].shape[0])

    for i in tqdm(range(len(all_fft_el))):
        mx = np.max(m) - all_fft_el[i].shape[0]

        zeros = np.zeros(shape=(mx, all_fft_el[i].shape[1]))
        zeros = pd.DataFrame(zeros, columns=all_fft_el[i].columns, index=np.arange(all_fft_el[i].shape[0], np.max(m)))
        
        all_fft_el[i] = all_fft_el[i].append(zeros)
        
        target = np.full(shape=(1, all_fft_el[i].shape[1]), fill_value = label)
        target = pd.DataFrame(target, columns=all_fft_el[i].columns, index=['target'])

        all_fft_el[i] = all_fft_el[i].append(target)
    
    all_fft = pd.DataFrame()
    
    for i in range(len(all_fft_el)):
        all_fft = pd.concat([all_fft, all_fft_el[i]], axis=1)

    return(all_fft)

In [12]:
def fft_creation_om(dataframe, label):
   
    _, all_fft_om = full_fft_list(dataframe)
    
    m = []
    
    for i in range(len(all_fft_om)):
        m.append(all_fft_om[i].shape[0])

    for i in tqdm(range(len(all_fft_om))):
        mx = np.max(m) - all_fft_om[i].shape[0]

        zeros = np.zeros(shape=(mx, all_fft_om[i].shape[1]))
        zeros = pd.DataFrame(zeros, columns=all_fft_om[i].columns, index=np.arange(all_fft_om[i].shape[0], np.max(m)))
        
        all_fft_om[i] = all_fft_om[i].append(zeros)

        target = np.full(shape=(1, all_fft_om[i].shape[1]), fill_value = label)
        target = pd.DataFrame(target, columns=all_fft_om[i].columns, index=['target'])

        all_fft_om[i] = all_fft_om[i].append(target)
    
    all_fft = pd.DataFrame()
    
    for i in range(len(all_fft_om)):
        all_fft = pd.concat([all_fft, all_fft_om[i]], axis=1)

    return(all_fft)

# Concat dataframes with spectrum for each class and download them

In [13]:
path = r'C:\Users\Public\Documents\HRS'

In [20]:
a = fft_creation_el(drivers, 1)
b = fft_creation_el(nondrivers, 0)
# c = fft_creation_el(noises, -1) 

final = pd.concat([a, b], axis=1)
final.to_csv(path + '\All FFT of EL no noise with hpf no zero del no 72_F75.csv', sep=',', index=True)

C:\Users\ecath\Anaconda3\lib\site-packages\pandas\core\frame.py:3035: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


(162, 10)
(165, 10)
(172, 8)
(175, 8)
(159, 14)
(162, 14)
(200, 14)
(203, 14)
(200, 14)
(203, 14)
(161, 22)
(163, 22)
(161, 22)
(163, 22)


100%|███████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 288.01it/s]


(162, 232)
(165, 232)
(172, 226)
(175, 226)
(159, 166)
(162, 166)
(200, 214)
(203, 214)
(200, 200)
(203, 200)
(161, 212)
(163, 212)
(161, 206)
(163, 206)


100%|███████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 160.36it/s]


In [21]:
a = fft_creation_om(drivers, 1)
b = fft_creation_om(nondrivers, 0)
# c = fft_creation_om(noises, -1) 

final = pd.concat([a, b], axis=1)
final.to_csv(path + '\All FFT of OM no noise with hpf no zero del no 72_F75.csv', sep=',', index=True)

C:\Users\ecath\Anaconda3\lib\site-packages\pandas\core\frame.py:3035: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


(162, 10)
(165, 10)
(172, 8)
(175, 8)
(159, 14)
(162, 14)
(200, 14)
(203, 14)
(200, 14)
(203, 14)
(161, 22)
(163, 22)
(161, 22)
(163, 22)


100%|███████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 229.44it/s]


(162, 232)
(165, 232)
(172, 226)
(175, 226)
(159, 166)
(162, 166)
(200, 214)
(203, 214)
(200, 200)
(203, 200)
(161, 212)
(163, 212)
(161, 206)
(163, 206)


100%|███████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 252.44it/s]
